In [1]:
import json
import csv
import os


input_path = "../raw_data/goodreads_books.json"
output_path = "../raw_data/goodreads_books_50k.csv"
max_rows = 50000


with open(input_path, "r", encoding="utf-8") as fin, \
     open(output_path, "w", encoding="utf-8", newline="") as fout:


    first_line = fin.readline().strip()
    first_obj = json.loads(first_line)

    fieldnames = list(first_obj.keys())
    writer = csv.DictWriter(fout, fieldnames=fieldnames)
    writer.writeheader()

    writer.writerow(first_obj)

    for i, line in enumerate(fin, start=2):
        if i > max_rows:
            break

        if not line.strip():
            continue

        obj = json.loads(line)
        writer.writerow(obj)


        if i % 10000 == 0:
            print(f"finished {i} rows...")

print(f"\nfinished {max_rows} rows → {output_path}")

finished 10000 rows...
finished 20000 rows...
finished 30000 rows...
finished 40000 rows...
finished 50000 rows...

finished 50000 rows → ../raw_data/goodreads_books_50k.csv


In [1]:
import pandas as pd
df = pd.read_csv("../raw_data/goodreads_books_50k.csv")

In [2]:
df.shape

(50000, 29)

In [3]:
df.columns.tolist()

['isbn',
 'text_reviews_count',
 'series',
 'country_code',
 'language_code',
 'popular_shelves',
 'asin',
 'is_ebook',
 'average_rating',
 'kindle_asin',
 'similar_books',
 'description',
 'format',
 'link',
 'authors',
 'publisher',
 'num_pages',
 'publication_day',
 'isbn13',
 'publication_month',
 'edition_information',
 'publication_year',
 'url',
 'image_url',
 'book_id',
 'ratings_count',
 'work_id',
 'title',
 'title_without_series']

In [4]:
df.head(3)

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
0,0312853122,1.0,[],US,NaN,"[{'count': '3', 'name': 'to-read'}, {'count': ...",NaN,False,4.00,NaN,...,9.0,NaN,1984.0,https://www.goodreads.com/book/show/5333265-w-...,https://images.gr-assets.com/books/1310220028m...,5333265,3.0,5400751.0,W.C. Fields: A Life on Film,W.C. Fields: A Life on Film
1,0743509986,6.0,[],US,NaN,"[{'count': '2634', 'name': 'to-read'}, {'count...",NaN,False,3.23,B000FC0PBC,...,10.0,Abridged,2001.0,https://www.goodreads.com/book/show/1333909.Go...,https://s.gr-assets.com/assets/nophoto/book/11...,1333909,10.0,1323437.0,Good Harbor,Good Harbor
2,NaN,7.0,['189911'],US,eng,"[{'count': '58', 'name': 'to-read'}, {'count':...",B00071IKUY,False,4.03,NaN,...,NaN,Book Club Edition,1987.0,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,7327624,140.0,8948723.0,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","The Unschooled Wizard (Sun Wolf and Starhawk, ..."


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   isbn                  29140 non-null  object 
 1   text_reviews_count    49991 non-null  float64
 2   series                50000 non-null  object 
 3   country_code          49992 non-null  object 
 4   language_code         27657 non-null  object 
 5   popular_shelves       50000 non-null  object 
 6   asin                  9935 non-null   object 
 7   is_ebook              49992 non-null  object 
 8   average_rating        49991 non-null  float64
 9   kindle_asin           21532 non-null  object 
 10  similar_books         50000 non-null  object 
 11  description           41352 non-null  object 
 12  format                36292 non-null  object 
 13  link                  49991 non-null  object 
 14  authors               50000 non-null  object 
 15  publisher          

In [6]:
missing_ratio = df.isna().mean().sort_values(ascending=False)
missing_ratio

edition_information     0.90880
asin                    0.80130
kindle_asin             0.56936
language_code           0.44686
publication_day         0.43354
isbn                    0.41720
publication_month       0.37222
isbn13                  0.32780
num_pages               0.32356
publisher               0.27712
format                  0.27416
publication_year        0.25422
description             0.17296
work_id                 0.00018
url                     0.00018
ratings_count           0.00018
link                    0.00018
average_rating          0.00018
text_reviews_count      0.00018
country_code            0.00016
is_ebook                0.00016
image_url               0.00016
similar_books           0.00000
series                  0.00000
popular_shelves         0.00000
authors                 0.00000
book_id                 0.00000
title                   0.00000
title_without_series    0.00000
dtype: float64

many information is missing in "edition_information", also not helpful, "asin" (Amazon ID, can delete), "kindle_asin" (Kindle Amazin ID), "publication_day", "publication_month", "format", "publisher", "isbn". Could be removed.

In [7]:
#description is important, 17% missing values, fillna with empty string
df["description"] = df["description"].fillna("")


In [8]:
df["language_code"].value_counts(dropna=False).head(20)

language_code
NaN      22343
eng      15218
en-US     2004
en-GB     1210
spa       1160
ita       1020
ara        855
fre        643
ger        607
ind        581
por        536
nl         375
tur        306
per        247
fin        245
swe        223
gre        215
cze        213
en-CA      189
jpn        160
Name: count, dtype: int64

In [11]:
#removed other languages books, keep only English and unknown language books
df["language_code_clean"] = df["language_code"].fillna("unknown")

english_codes = ["eng", "en-US", "en-GB", "en-CA", "unknown"]

df_eng = df[df["language_code_clean"].isin(english_codes)].copy()
df_eng=df_eng.drop(columns=["language_code"])
df_eng.shape

(40964, 29)

In [12]:
df_eng["authors"].head(3)

0    [{'author_id': '604031', 'role': ''}]
1    [{'author_id': '626222', 'role': ''}]
2     [{'author_id': '10333', 'role': ''}]
Name: authors, dtype: object

In [13]:
# changing formats for "Authors", "Popular Shelves", "Similar Books" columns
import ast
def parse_list_dict(s):
    try:
        return ast.literal_eval(s)
    except:
        return []

In [14]:
df_eng["authors_parsed"] = df_eng["authors"].apply(parse_list_dict)
df_eng["author_ids"] = df_eng["authors_parsed"].apply(lambda lst: [d.get("author_id") for d in lst])

In [15]:
df_eng["shelves_parsed"] = df_eng["popular_shelves"].apply(parse_list_dict)
df_eng["shelf_names"] = df_eng["shelves_parsed"].apply(lambda lst: [d.get("name") for d in lst])

In [16]:
def parse_simple_list(s):
    try:
        return ast.literal_eval(s)
    except:
        return []

df_eng["similar_books_parsed"] = df_eng["similar_books"].apply(parse_simple_list)

In [17]:
df_eng.shape

(40964, 34)

In [18]:
df_eng=df_eng.drop(columns=["authors", "authors_parsed", "popular_shelves", "shelves_parsed", "similar_books"])

In [19]:
# To remove columns that are not useful for analysis
cols_to_drop = [
    "edition_information",
    "asin",
    "kindle_asin",
    "publication_day",
    "publication_month",
    "format",
    "publisher",
    "isbn"          # kept isbn13,
    "authors", "authors_parsed", "popular_shelves", "shelves_parsed", "similar_books","language_code"
]


In [22]:
df_eng=df_eng.drop(columns=cols_to_drop)

KeyError: "['isbnauthors', 'authors_parsed', 'popular_shelves', 'shelves_parsed', 'similar_books', 'language_code'] not found in axis"

### Not yet droped columns

In [23]:
df_eng = df_eng.reset_index(drop=True)

In [24]:
import re

In [25]:
df_eng["title_clean"] = (
    df_eng["title_without_series"]
    .fillna(df_eng["title"])
    .fillna("")
    .astype(str)
    .str.lower()
)

In [ ]:
def clean_description(text):
    if not isinstance(text, str):
        return ""
    # lower case
    text = text.lower()
    # revove simple HTML tags
    text = re.sub(r"<.*?>", " ", text)
    # remove newlines and tabs
    text = text.replace("\n", " ").replace("\r", " ").replace("\t", " ")
    # remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [28]:
df_eng["description_clean"] = df_eng["description"].fillna("").astype(str).apply(clean_description)

In [30]:
df_eng[["title", "title_clean", "description"]].head()


,title,title_clean,description
0,W.C. Fields: A Life on Film,w.c. fields: a life on film,
1,Good Harbor,good harbor,"Anita Diamant's international bestseller ""The ..."
2,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","the unschooled wizard (sun wolf and starhawk, ...",Omnibus book club edition containing the Ladie...
3,Best Friends Forever,best friends forever,Addie Downs and Valerie Adler were eight when ...
4,Runic Astrology: Starcraft and Timekeeping in ...,runic astrology: starcraft and timekeeping in ...,


In [31]:
# create combined text for each book for similarity and clustering

def build_combined_text(row):
    parts = []


    if isinstance(row["title_clean"], str):
        parts.append(row["title_clean"])


    if isinstance(row["description_clean"], str):
        parts.append(row["description_clean"])


    return " ".join(parts)

In [32]:
df_eng["combined_text"] = df_eng.apply(build_combined_text, axis=1)

In [33]:
df_eng["combined_text"].head(3)

0                         w.c. fields: a life on film 
1    good harbor anita diamant's international best...
2    the unschooled wizard (sun wolf and starhawk, ...
Name: combined_text, dtype: object

In [34]:
#TF-IDF Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


In [36]:
vectorizer = TfidfVectorizer(
    stop_words="english",   # remove English common words (the, and, of...)
    max_features=20000,     # only keep the top 20,000 "informative" words
    min_df=5,               # at least appear in 5 books
    max_df=0.8              # if a word appears in more than 80% of books, consider it uninformative
)

In [37]:
tfidf_matrix = vectorizer.fit_transform(df_eng["combined_text"])

tfidf_matrix.shape

(40964, 20000)

In [38]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
def get_similar_books_by_index(book_idx, top_n=5):
    """
    input：book_idx，output：the most similar top_n books
    """
    # 1. get the vector for the given book
    book_vec = tfidf_matrix[book_idx]   # 1 x N

    # 2. compute cosine similarity with all books
    sim_scores = cosine_similarity(book_vec, tfidf_matrix).ravel()

    # 3. sort and remove self
    # argsort from small to large, we take the last top_n+1, then remove self
    top_indices = np.argsort(sim_scores)[- (top_n + 1) :][::-1]  # from large to small
    top_indices = [idx for idx in top_indices if idx != book_idx][:top_n]

    # 4. return a DataFrame
    result = df_eng.loc[top_indices, ["title", "average_rating", "ratings_count"]].copy()
    result["similarity"] = sim_scores[top_indices]
    return result

In [42]:
get_similar_books_by_index(9, top_n=5)

,title,average_rating,ratings_count,similarity
6065,Exploring the Occult,3.25,7.0,0.211142
15454,The Devil,3.50,2.0,0.206279
19411,Devil's Marionette,3.83,12.0,0.206279
15831,The Devil's Vial,3.88,6.0,0.206279
1100,The Devil in Maryvale (Maryvale #1),3.88,61.0,0.206279


In [43]:
from sklearn.cluster import MiniBatchKMeans


In [ ]:
n_clusters = 20
kmeans = MiniBatchKMeans(
    n_clusters=n_clusters,
    random_state=42,
    batch_size=1000
)

cluster_labels = kmeans.fit_predict(tfidf_matrix)

df_eng["cluster"] = cluster_labels

In [45]:
df_eng["cluster"].value_counts().sort_index()


cluster
0         1
1         1
2         1
3         2
4         5
5         1
6         1
7         1
8         1
9         1
10    40933
11        1
12        1
13        1
14        1
15        8
16        1
17        1
18        1
19        1
Name: count, dtype: int64

In [46]:
def show_cluster_examples(c, n=10):
    subset = df_eng[df_eng["cluster"] == c]
    return subset[["title", "average_rating", "ratings_count"]].head(n)

show_cluster_examples(0)

,title,average_rating,ratings_count
18646,Confessions of an Instinctively Mutinous Baby ...,4.31,142.0
